In [1]:
import requests
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

df = pd.read_csv("scaledlist_5000_AB300 Test List 10_27_25.csv")
base_url = "https://earthquake.usgs.gov/ws/designmaps/asce7-22.json"
params_template = {
    "riskCategory": "III",
    "siteClass": "Default",
    "title": "Example"
}
# param
results = []
failed_rows_722 = []
max_workers = 6       # Number of parallel threads
delay = 0.5           # Delay per request to prevent overloading API

In [4]:


def usgs_fetch(row, params_template, base_url, max_retries=3, delay=0.5):
    params = {
        **params_template,
        "latitude": row["Facility latitude"],
        "longitude": row["Facility Longitude"]
    }

    for attempt in range(max_retries):
        try:
            response = requests.get(base_url, params=params, timeout=10)
            if response.status_code == 200:
                data = response.json()
                seismic = data.get("response", {}).get("data", {})
                sms = seismic.get("sms")
                sm1 = seismic.get("sm1")
                ss = seismic.get("ss")
                s1 = seismic.get("s1")
                sdc = seismic.get("sdc")
                
                #print
                print(f"{row['FacilityName']} -> SMS: {sms}, SM1: {sm1}, SS: {ss}, S1: {s1}, SDC: {sdc}")

                

                return {
                    "School ID": row["cds_code"],
                    "Facility": row["FacilityName"],
                    "Latitude": row["Facility latitude"],
                    "Longitude": row["Facility Longitude"],
                    "SMS": sms,
                    "SM1": sm1,
                    "SS": ss,
                    "S1": s1,
                    "SDC": sdc,
                }

            elif response.status_code == 429:
                # Rate limit hit
                print(f"Rate limit hit for {row['FacilityName']} (attempt {attempt + 1}) — sleeping 60 seconds")
                time.sleep(60)
                continue
            else:
                print(f"Attempt {attempt + 1} failed for {row['FacilityName']} with status {response.status_code}")

        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} exception for {row['FacilityName']}: {e}")

        time.sleep(delay)

    # If all retries fail, return placeholder and mark as failed
    failed_rows_722.append({
        "School ID": row["cds_code"],
        "Facility": row["FacilityName"],
        "Latitude": row["Facility latitude"],
        "Longitude": row["Facility Longitude"]
    })

    return {
        "School ID": row["cds_code"],
        "Facility": row["FacilityName"],
        "Latitude": row["Facility latitude"],
        "Longitude": row["Facility Longitude"],
        "SMS": None,
        "SM1": None,
        "SS": None,
        "S1": None,
        "SDC": None,
    }



In [5]:

# Run concurrent 
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(usgs_fetch, row, params_template, base_url) for _, row in df.iterrows()]
    for future in as_completed(futures):
        results.append(future.result())

#sds_results.sort(key=lambda x: x['Facility'])


Academy of the Redwoods -> SMS: 3.55, SM1: 2.45, SS: 3.38, S1: 1.18, SDC: E
Adams Middle -> SMS: 2.75, SM1: 2.16, SS: 2.62, S1: 1.02, SDC: E
Alameda County Juvenile Hall/Court -> SMS: 2.54, SM1: 1.87, SS: 2.44, S1: 0.86, SDC: E
Acacia Middle -> SMS: 2.52, SM1: 2.27, SS: 2.37, S1: 0.87, SDC: E
Abraham Lincoln Elementary -> SMS: 2.87, SM1: 2.59, SS: 2.68, S1: 0.98, SDC: E
Academy of the Canyons -> SMS: 2.72, SM1: 1.92, SS: 2.59, S1: 0.91, SDC: E
Alice Birney Elementary -> SMS: 2.9, SM1: 2.63, SS: 2.72, S1: 0.99, SDC: E
Alder Middle -> SMS: 2.57, SM1: 2.32, SS: 2.41, S1: 0.88, SDC: E
Alice Birney Elementary -> SMS: 3.64, SM1: 2.46, SS: 3.52, S1: 1.19, SDC: E
Allendale Elementary -> SMS: 2.54, SM1: 1.95, SS: 2.41, S1: 0.85, SDC: E
Alessandro High -> SMS: 2.47, SM1: 2.19, SS: 2.31, S1: 0.83, SDC: E
Alexander Rose Elementary -> SMS: 2.69, SM1: 2.04, SS: 2.57, S1: 0.99, SDC: E
American Indian Public Charter -> SMS: 2.64, SM1: 2.03, SS: 2.52, S1: 0.89, SDC: EAlternative Learning Academy at Con

In [6]:
#results.sort(key=lambda x: x['Facility'])
# Save to CSV
pd.DataFrame(results).to_csv("Default_Risk_III_Schools_sms.csv", index=False)
pd.DataFrame(failed_rows_722).to_csv("failed_rows_Default_III_sms.csv", index=False)
